# Экзаменационное проектное задание

## Шкала оценивания

- **Оценка 3-4:** Нужно построить модель машинного обучения с классификатором по заданию

- **Оценка 5:** Нужно применить 2 дополнительных классификатора, сравнить и сделать выводы какой лучше

# Часть I. Подготовка набора данных: Отзывы о ресторанах

## Задание

Вам предлагается выполнить подготовку набора данных

Описание набора данных: Отзывы пользователей о ресторанах. Целевая переменная - тональность отзыва

Ссылка на набор данных для использования в блокноте: https://raw.githubusercontent.com/yakushinav/mo2025/refs/heads/main/data/rest01.csv

#### 1. Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import resample

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

#### 2. Чтение набора данных

In [2]:
url = "https://raw.githubusercontent.com/yakushinav/mo2025/refs/heads/main/data/rest01.csv"
df = pd.read_csv(url)


#### 3. Первые 7 строк набора данных

In [3]:
df.head(7)


,Unnamed: 0,rest,review,feedback
0,0,13 маршрут Retro-Blues,"День 8-го марта прошёл, можно и итоги подвести...",positive
1,1,Веселый барин,Отмечали в этом ресторане день рождение на пер...,positive
2,2,Моб Джойнт,Для встречи с друзьями было выбрано данное зав...,neutral
3,3,Крапива,Хочу поделиться своим впечатлением от посещени...,negative
4,4,"Советское кафе ""Квартирка""",Добрый день! Были вчера с друзьями в этом кафе...,positive
5,5,Дитай,Отметили с мужем годовщину свадьбы 6 ноября в ...,neutral
6,6,ПАБ № 1,Впервые побывала в этом пабе совсем недавно и ...,neutral


#### 4. Последние 5 строк набора данных

In [4]:
df.tail(5)


,Unnamed: 0,rest,review,feedback
399,399,Neverland,Пришли в данное заведение 4 июня 2014 года пок...,negative
400,400,The kitchen,Заехали с мужем поужинать в пятницу ( 17.01.14...,positive
401,401,Доски,"Пришел сегодня с друзьями, отметить день рожде...",neutral
402,402,Шатер,Мне так там нравитсяяяя!!!!!!!!! Интерьер модн...,positive
403,403,Рица,Уютная и тёплая домашняя обстановка! Милый и о...,positive


#### 5. Поля набора данных

In [5]:
df.columns


Index(['Unnamed: 0', 'rest', 'review', 'feedback'], dtype='object')

#### 6. Размер набора данных (количество полей и строк)

In [6]:
df.shape

(404, 4)

#### 7. Опишите поля набора данных в формате: название поля, тип данных, назначение поля

In [7]:
print("Описание полей набора данных:")
print("=" * 60)
for col in df.columns:
    dtype = df[col].dtype
    if col == 'Unnamed: 0':
        purpose = "Идентификатор записи"
    elif col == 'rest':
        purpose = "Название ресторана"
    elif col == 'review':
        purpose = "Текст отзыва посетителя"
    elif col == 'feedback':
        purpose = "Тональность отзыва (целевая переменная)"
    else:
        purpose = "Неопределенное назначение"

    print(f"Поле: {col}")
    print(f"Тип данных: {dtype}")
    print(f"Назначение: {purpose}")
    print("-" * 40)

Описание полей набора данных:
Поле: Unnamed: 0
Тип данных: int64
Назначение: Идентификатор записи
----------------------------------------
Поле: rest
Тип данных: object
Назначение: Название ресторана
----------------------------------------
Поле: review
Тип данных: object
Назначение: Текст отзыва посетителя
----------------------------------------
Поле: feedback
Тип данных: object
Назначение: Тональность отзыва (целевая переменная)
----------------------------------------


#### 8. Информация о наборе данных

In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  404 non-null    int64 
 1   rest        404 non-null    object
 2   review      404 non-null    object
 3   feedback    404 non-null    object
dtypes: int64(1), object(3)
memory usage: 12.8+ KB


#### 9. Проверка наличия пропусков в данных

In [9]:
df.isnull().sum()


,0
Unnamed: 0,0
rest,0
review,0
feedback,0


#### 10. Если вы обнаружили пропуски в данных, то удалите их

In [10]:
# не было

#### 11. Проведите предобработку текстовых данных: удаление символов, лемматизация, стоп слова, перевод в нижний регистр

In [11]:
mystem = Mystem()
russian_stopwords = set(stopwords.words("russian"))

def preprocess_text(text):
    if pd.isna(text):
        return ""

    # нижний регистр
    text = text.lower()

    # Удаление лишних символов
    text = re.sub(r"[^а-яё\s]", " ", text)

    # Удаляем множественные пробелы (на всякий случай)
    text = re.sub(r"\s+", " ", text).strip()

    # Токенизация
    tokens = word_tokenize(text, language="russian")

    # Удаляем стоп-слова и короткие слова
    tokens = [token for token in tokens if token not in russian_stopwords and len(token) > 2]

    # Лемматизация
    if tokens:
        lemmas = mystem.lemmatize(" ".join(tokens))
        lemmas = [lemma.strip() for lemma in lemmas if lemma.strip() and len(lemma.strip()) > 1]
        return " ".join(lemmas)

    return ""

# Применяем предобработку
df = df.dropna(subset=['review'])
df['processed_review'] = df['review'].apply(preprocess_text)
df = df[df['processed_review'].str.len() > 0]


Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


#### 12. Сделайте вывод о пригодности набора данных для построения модели машинного обучения

In [12]:
print("В целом можно сказать, что набор нормальный, хотя и несбалансированный. После очистки и лемматизации данные пригодны для обучения модели классификации.")


В целом можно сказать, что набор нормальный, хотя и несбалансированный. После очистки и лемматизации данные пригодны для обучения модели классификации.


# Часть II. Построение модели машинного обучения для набора данных: Отзывы о ресторанах

## Задание

Вам нужно решить задачу классификации с помощью алгоритма

Случайный лес RandomForestClassifier



Целевая переменная, результат: **feedback**

#### 13. Разделить выборку на признаки (Х) и результат (Y)

In [13]:
print("Распределение классов:")
print(df['feedback'].value_counts())

min_class_size = df['feedback'].value_counts().min()

balanced_dfs = []
for class_label in df['feedback'].unique():
    class_df = df[df['feedback'] == class_label]
    if len(class_df) > min_class_size:
        class_df = resample(class_df, n_samples=min_class_size, random_state=42)
    balanced_dfs.append(class_df)

df_balanced = pd.concat(balanced_dfs, ignore_index=True)

print("\nРаспределение после балансировки:")
print(df_balanced['feedback'].value_counts())

X = df_balanced['processed_review']
y = df_balanced['feedback']

print(f"\nРазмеры: X={len(X)}, y={len(y)}")


Распределение классов:
feedback
positive    279
neutral      84
negative     41
Name: count, dtype: int64

Распределение после балансировки:
feedback
positive    41
neutral     41
negative    41
Name: count, dtype: int64

Размеры: X=123, y=123


#### 14. Разделить на обучающую и тестовую выборки

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Обучающая выборка: {len(X_train)}")
print(f"Тестовая выборка: {len(X_test)}")
print(f"Распределение в обучающей выборке:\n{y_train.value_counts()}")


Обучающая выборка: 98
Тестовая выборка: 25
Распределение в обучающей выборке:
feedback
neutral     33
negative    33
positive    32
Name: count, dtype: int64


#### 15. TF-IDF векторизация и мешок слов

In [15]:
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    max_df=0.8,
    min_df=3,
    max_features=5000,
    sublinear_tf=True,
    use_idf=True,
    smooth_idf=True
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(f"Размер матрицы признаков: {X_train_vec.shape}")
print(f"Количество уникальных слов: {len(vectorizer.vocabulary_)}")


Размер матрицы признаков: (98, 988)
Количество уникальных слов: 988


#### 16. Сформировать модель машинного обучения

In [16]:
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

print("Модель RandomForest создана с оптимизированными параметрами")


Модель RandomForest создана с оптимизированными параметрами


#### 17. Обучить модель

In [17]:
rf_model.fit(X_train_vec, y_train)
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(rf_model, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train_vec, y_train)
best_rf = grid_search.best_estimator_


#### 18. Оценить качество модели

In [18]:
y_pred = rf_model.predict(X_test_vec)
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))


              precision    recall  f1-score   support

    negative       0.64      0.88      0.74         8
     neutral       0.60      0.38      0.46         8
    positive       0.56      0.56      0.56         9

    accuracy                           0.60        25
   macro avg       0.60      0.60      0.58        25
weighted avg       0.60      0.60      0.58        25

Accuracy: 0.6


#### 19. Выполнить предсказание класса для трех разных фраз

In [19]:
examples = [
    "Очень понравилось обслуживание и еда!",
    "Ужасное место, плохо, больше не приду.",
    "Средний ресторан, ничего особенного."
]
examples_proc = [preprocess_text(text) for text in examples]
examples_vec = vectorizer.transform(examples_proc)
preds = rf_model.predict(examples_vec)
for phrase, pred in zip(examples, preds):
    print(f"Фраза: {phrase} — Класс: {pred}")


Фраза: Очень понравилось обслуживание и еда! — Класс: positive
Фраза: Ужасное место, плохо, больше не приду. — Класс: neutral
Фраза: Средний ресторан, ничего особенного. — Класс: neutral


#### 20. По итогам сделать вывод о качестве и пригодности модели машинного обучения для использования

In [20]:
print("Модель RandomForestClassifier показала адекватные результаты на тестовой выборке. Классификатор более-менее подходит для автоматического определения тональности отзывов о ресторанах.")


Модель RandomForestClassifier показала адекватные результаты на тестовой выборке. Классификатор пригоден для автоматического определения тональности отзывов о ресторанах.


In [21]:
logreg = LogisticRegression(
    penalty='l1',
    solver='liblinear',
    C=0.5,
    max_iter=1000,
    class_weight='balanced'
)
logreg.fit(X_train_vec, y_train)


LogisticRegression(C=0.5, class_weight='balanced', max_iter=1000, penalty='l1',
                   solver='liblinear')

#### 21. Постройте еще две модели машинного обучения, сравните той, что была в задании и сделайте вывод о том, какая модель лучше

In [22]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

# Создаем дополнительные модели
models = {
    'RandomForest': rf_model,
    'LogisticRegression': LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        random_state=42
    ),
    'MultinomialNB': MultinomialNB(alpha=0.1),
    'SVM': SVC(
        kernel='linear',
        class_weight='balanced',
        random_state=42,
        probability=True
    )
}

print("Сравнение моделей:")
print("=" * 60)

best_model = None
best_score = 0

for name, model in models.items():
    # Обучаем модель
    model.fit(X_train_vec, y_train)

    cv_scores = cross_val_score(model, X_train_vec, y_train, cv=5, scoring='f1_macro')

    # Тестирование
    y_pred = model.predict(X_test_vec)
    test_accuracy = accuracy_score(y_test, y_pred)

    print(f"{name}:")
    print(f"  CV F1-macro: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    print(f"  Test Accuracy: {test_accuracy:.3f}")
    print(f"  Classification Report:")
    print(classification_report(y_test, y_pred))
    print("-" * 60)

    # Отслеживаем лучшую модель
    if test_accuracy > best_score:
        best_score = test_accuracy
        best_model = (name, model)

print(f"\nЛучшая модель: {best_model[0]} с точностью {best_score:.3f}")


Сравнение моделей:
RandomForest:
  CV F1-macro: 0.621 ± 0.076
  Test Accuracy: 0.600
  Classification Report:
              precision    recall  f1-score   support

    negative       0.64      0.88      0.74         8
     neutral       0.60      0.38      0.46         8
    positive       0.56      0.56      0.56         9

    accuracy                           0.60        25
   macro avg       0.60      0.60      0.58        25
weighted avg       0.60      0.60      0.58        25

------------------------------------------------------------
LogisticRegression:
  CV F1-macro: 0.652 ± 0.046
  Test Accuracy: 0.640
  Classification Report:
              precision    recall  f1-score   support

    negative       0.73      1.00      0.84         8
     neutral       0.50      0.25      0.33         8
    positive       0.60      0.67      0.63         9

    accuracy                           0.64        25
   macro avg       0.61      0.64      0.60        25
weighted avg       0.61  